# 

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import re

In [3]:
# df utility
def fColumns(df):
    print(pd.DataFrame.from_dict({ 'column': df.columns.values, 
                            'dtype': df.dtypes.values,
                            'nmissing': df.isna().apply(sum).values}))
    return

In [4]:
## all yearly data
# the raw csv have been converted to pipe-separated by csvtool for convenience
files = [ 'SFF_2016-2019.psv', 'SFF_2020-2023.psv', 'SFF_2024.psv' ]
fpath = 'data/prep/'

In [5]:
## to read the input files
def read_psv(f):
    return pd.read_csv(f, sep='|', header=0
        # , nrows=1000
    , dtype={ 'Id': np.int32, 'Course': pd.StringDtype(), 'Race': pd.StringDtype()
            , 'Type': pd.StringDtype(), 'AgeLimit': pd.StringDtype()
            , 'Prize': np.float64, 'Ran': np.int8, 'Distance': pd.StringDtype(), 'Yards': np.int16
            , 'Going': pd.StringDtype()
        , 'WinTime': pd.StringDtype() # redundant CHECK
        , 'Seconds': np.float64
        , 'FPos': pd.StringDtype(), 'DstBtn': pd.StringDtype(), 'TotalBtn': pd.StringDtype()
        # , 'CardNo': np.int8   # has NAs
        , 'HorseName': pd.StringDtype()
        # , 'Draw': np.int8   # has NAs
        , 'Sp': np.float64, 'Age': np.int8
        , 'Stone': np.int8, 'Lbs': np.int8
        , 'WeightLBS': np.int16 # redundant CHECK
        , 'Fav': pd.StringDtype(), 'Aid': pd.StringDtype()
        , 'Trainer': pd.StringDtype(), 'Jockey': pd.StringDtype()
        # , 'Allow': np.int8   # has NAs
        , 'OR': pd.StringDtype() # np.int16 with NAs but throws "mixed types" warning if unspecified
        , 'Comments': pd.StringDtype()
        }
    , parse_dates=[2, 3]    # works but throws a warning for RaceDate as 'd-Mon-yy'
    , dayfirst=True         # owing to inconsistent formats
    , thousands=',' # or null
    , encoding='cp1252')

In [6]:
df = pd.concat([ read_psv(fpath + f) for f in files ]).reset_index(drop=True)

/tmp/ipykernel_72994/2175177951.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(f, sep='|', header=0
/tmp/ipykernel_72994/2175177951.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(f, sep='|', header=0
/tmp/ipykernel_72994/2175177951.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(f, sep='|', header=0
/tmp/ipykernel_72994/2175177951.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please spec

In [7]:
## deal with each column in turn

In [8]:
# ID -> RaceID ... as is
df.rename({ 'Id': 'RaceId' }, axis=1, inplace=True)

In [9]:
# Course -> CourseName ...
df.rename({ 'Course': 'CourseName' }, axis=1, inplace=True)

In [10]:
def fTerritory(x):
    y = re.search(r' \(([A-Z]+)\)', x)
    return y.group(1) if y else 'GB'

In [11]:
# list(df.CourseName.unique())

In [12]:
# list(df.CourseName.apply(fTerritory).unique())

In [13]:
df['Territory'] = df.CourseName.apply(fTerritory)
df.loc[df.Territory=='NI', 'Territory'] = 'IRE' # n. NI prizes are quoted in euro
# remove non-GB/IRE races [not in 2024 data]
df = df[(df.Territory=='GB') | (df.Territory=='IRE')]
# drop the suffix as this is now in Territory
df['CourseName'] = df['CourseName'].apply(lambda x: x.removesuffix(' (NI)').removesuffix(' (IRE)'))

In [14]:
# RaceDate ...
# RaceTime ...

In [15]:
""" https://stackoverflow.com/questions/61330414/pandas-astype-with-date-or-datetime/75990548#75990548
"""
df['RaceDate'] = df['RaceDate'].dt.date
df['RaceTime'] = df['RaceTime'].dt.time

In [16]:
# Race ...
df['Md'] = df.Race.str.contains('Maiden')
df['Nv'] = df.Race.str.contains('Novice')
# 'Restricted' ...
df['Hc'] = df.Race.str.contains('Handicap')

df['RT1'] = df.apply(lambda r: 'Md' if r.Md else\
                              'NvHc' if r.Nv & r.Hc else\
                              'Nv' if r.Nv else\
                              'Hc' if r.Hc else '', axis=1)

In [17]:
# tidy up
df.drop(['Md','Nv','Hc'], axis=1, inplace=True)

In [18]:
df['F'] = df.Race.str.contains('Fillies')
df['M'] = df.Race.str.contains('Mares')
# df['CG'] = df.Race.str.contains('Colts & Geldings') | df.Race.str.contains('Colts And Geldings')
# df['C'] = df.Race.str.contains('\(Colts\)')
# df['CF'] = df.Race.str.contains('Colts & Fillies') | df.Race.str.contains('Colts And Fillies')
# df['NG'] = df.Race.str.contains('No Geldings')

df['RT2'] = df.apply(lambda r: 'M' if r.M else\
                              'F' if r.F else '', axis=1)

In [19]:
# tidy up
df.drop(['F','M'], axis=1, inplace=True)

In [20]:
# tidy up
df.drop('Race', axis=1, inplace=True)

In [21]:
# Type ...

In [22]:
df['H'] = ~df.Type.isna() & (df.Type == 'h')
df['Ch'] = ~df.Type.isna() & (df.Type == 'c')
df['NHF'] = ~df.Type.isna() & (df.Type == 'b')
# df['Flat'] = df.Type.isna()

df['RT3'] = df.apply(lambda r: 'H' if r.H else\
                               'Ch' if r.Ch else\
                               'NHF' if r.NHF else '', axis=1)

In [23]:
# tidy up
df.drop(['H','Ch','NHF'], axis=1, inplace=True)

In [24]:
# v2.0
# tidy up
# df.drop('Type', axis=1, inplace=True)

In [25]:
df['RT'] = df.apply(lambda r: r.RT1 + r.RT3 + r.RT2, axis=1)

In [26]:
# tidy up
df.drop(['RT1','RT2','RT3'], axis=1, inplace=True)

In [27]:
# key variable for categorising races
# df.RT.value_counts()

In [28]:
# Class ...

In [29]:
# Class 47 Dun 14/02/24 ... Class 50 Gal .. corrections
df.loc[(df.Class==47)|(df.Class==50), 'Class'] = np.nan

In [30]:
# Agelimit ...

In [31]:
# parse AgeLimit string to numeric min and max
df['AgeMin'] = [ float(x[:x.find('Y')]) for x in df.AgeLimit ]
df['AgeMax'] = [ float(x[:x.find('Y')]) if x.endswith('only') 
                else float(x[x.rfind('Y')-1]) if x.find('to') > 0    # < 10
                else np.nan for x in df.AgeLimit ]

In [32]:
# tidy up
df.drop('AgeLimit', axis=1, inplace=True)

In [33]:
# Prize ...
# TODO parse to numeric

In [34]:
# Ran -> FieldSize ... as is
df.rename({ 'Ran': 'FieldSize' }, axis=1, inplace=True)

In [35]:
# Distance ... redundant
df.drop('Distance', axis=1, inplace=True)

In [36]:
# Yards -> Distance ... as is
df.rename({ 'Yards': 'Distance' }, axis=1, inplace=True)

In [37]:
# Going ...

In [38]:
# get AW from Going as not in Course
# (could also get tapeta/polytrack/sand here but we use separate lookup instead)
df['AW'] = df.Going.str.contains('AW')

In [39]:
# # parse the Going to primary and secondary ('in places', etc.)
# def fGoing(x):
#     return x.replace('AW','').lower().replace(' ','').replace('-','').replace('sand','').\
#                 replace('tapeta','').replace('polytrack','').\
#                 replace('standard/slow','goodtosoft').\
#                 replace('standard','good').\
#                 replace('watered','').replace('watering','').\
#                 replace('straight','').replace('home','').replace('back','').\
#                 replace('downhillsection','').replace('adjacenttolake','').replace('bend','').\
#                 replace('places','').replace('(str)','').replace('lastmile','').\
#                 replace('5fchute','').replace('last3f','').\
#                 replace('(','|').replace(',','|').replace(')','').\
#                 replace('goyieldingodtoyielding','yielding').\
#                 replace('goodtioyielding','goodtoyielding').\
#                 replace('goodtofim','goodtofirm').\
#                 replace('few','').replace('','').\
#                 replace('in','').replace('on','').replace('and','').replace('the','')

In [40]:
# parse the Going to primary (as this is all that will be available after 31 mar 25, ie from RP not RF)
def fGoing(x):
    return x.replace('AW','').lower().replace('fibresand','').replace('sand','').\
            replace('tapeta','').replace('polytrack','').\
            replace('(','|').replace(',','|').\
            replace(' ','').\
            partition('|')[0].\
            replace('roundgood;straightgoodtoyielding','good').\
            replace('hurdle:good-yieldinginplaces;chase:yielding','yielding').\
            replace('hurdle:good-goodtoyieldinginplaces;chase:goodtoyielding','goodtoyielding').\
            partition('-')[0]

In [41]:
df['Going1'] = df.Going.apply(fGoing)

In [42]:
GoingDict = {'firm': 1.0, 'goodtofirm': 2.0, 'standard/fast': 2.0, 'good': 3.0,
             'standard': 3.0, 'goodtoyielding': 3.5, 'yielding': 4.0,
             'goodtosoft': 4.0, 'standard/slow': 4.0, 'yieldingtosoft': 4.5,
             'soft': 5.0, 'slow': 5.0, 'softtoheavy': 5.5, 'heavy': 6.0}

In [43]:
df['Going1N'] = df.Going1.apply(lambda x: GoingDict[x])

In [44]:
# # eyeball the results
# a = pd.DataFrame({'Going': df.Going.unique()})
# a['Going1'] = a.Going.apply(fGoing)
# a['GoingN'] = a.Going1.apply(lambda x: GoingDict[x])
# pd.set_option('display.max_rows', 500)
# print(a)
# pd.set_option('display.max_rows', 50)

In [45]:
# tidy up
df.drop('Going', axis=1, inplace=True)

In [46]:
# Limit ... not used
df.drop('Limit', axis=1, inplace=True)

In [47]:
# WinTime ... redundant
df.drop('WinTime', axis=1, inplace=True)

In [48]:
# Seconds -> RaceWinTime ... as is
df.rename({ 'Seconds': 'RaceWinTime' }, axis=1, inplace=True)

In [49]:
# FPos -> Posn ... 
df.rename({ 'FPos': 'Posn' }, axis=1, inplace=True)

In [50]:
# # Posn <NA> implies did not finish (6 rows)
# df[df.Posn.isna()]['Comments'].values
df.loc[df.Posn.isna(), 'Posn'] = '-'

In [51]:
df['Won'] = (df.Posn=='1')

In [52]:
# # not all races have winners (eg abandoned, no finishers)
# df.Won.sum() - len(df.RaceId.unique())

In [53]:
# https://www.justbookies.com/betting-explained/place-terms-explained/
#     1-4 runners: win only.
#     5-7 runners: 1/4 odds a place for 1,2.
#     8+ runners: 1/5 odds a place for 1,2,3.
#     12-15 runners in handicaps only: 1/4 odds a place 1,2,3
#     16+ runners in handicaps only: 1/4 odds a place 1,2,3,4.
df['Placed'] = df.apply(lambda r: 0 if r.FieldSize < 5 else\
                                  1/4 if r.FieldSize < 8 and r.Posn in ['1','2'] else\
                                  1/5 if r.FieldSize < 12 and r.Posn in ['1','2','3'] else\
                                  1/5 if r.FieldSize >= 12 and r.Posn in ['1','2','3'] and r.RT.find('Hc')==-1 else\
                                  1/4 if r.FieldSize < 16 and r.Posn in ['1','2','3'] and r.RT.find('Hc')>-1 else\
                                  1/4 if r.FieldSize >= 16 and r.Posn in ['1','2','3','4'] and r.RT.find('Hc')>-1 else\
                                  0, axis=1)

In [54]:
# DstBtn ... not used
df.drop('DstBtn', axis=1, inplace=True)

In [55]:
# TotalBtn ...
df.loc[df.Posn=='1', 'TotalBtn'] = '0'  # some of these are originall 'Null'
df.loc[df.TotalBtn=='Null', 'TotalBtn'] = pd.NA  # did not finish
df['TotalBtn'] = df['TotalBtn'].astype('float32')

In [56]:
# CardNo ... irrelevant
df.drop('CardNo', axis=1, inplace=True)

In [57]:
# HorseName ...
#
# ... includes Territory ...
# set(''.join(list(df.HorseName.unique())))
# df.HorseName[df.HorseName.str.find('(')>-1]
# df.HorseName.apply(fTerritory).unique()

In [58]:
# Draw ... as is

In [59]:
# Sp ...
df['ReturnWin'] = df['Won'].astype(int) * (1 + df.Sp)
df['ReturnPlace'] = (df['Placed']>0).astype(int) * (1 + df.Sp*df.Placed)

In [60]:
# v2.0
# tidy up
# df.drop('Sp', axis=1, inplace=True)

In [61]:
# Age ... not used
df.drop('Age', axis=1, inplace=True)

In [62]:
# Stone, Lbs, WeightLBS ...

In [63]:
df['WeightLBS'] = df.Stone.astype('int16')*14 + df.Lbs.astype('int16')

In [64]:
# tidy up
df.drop(['Stone','Lbs'], axis=1, inplace=True)

In [65]:
# Favs ... not used
df.drop('Favs', axis=1, inplace=True)

In [66]:
# v2.0
# Aid ... not used
# df.drop('Aid', axis=1, inplace=True)

In [67]:
# Trainer ...
df.loc[df.Trainer.isna(), 'Trainer'] = 'NK'

In [68]:
# Jockey ...
df.loc[df.Jockey.isna(), 'Jockey'] = 'NK'

In [69]:
# df[df.Jockey=='Miss Polly Gundry']

In [70]:
# Allow ...
df.loc[df.Allow.isna(), 'Allow'] = 0
df.Allow = df.Allow.astype(int)

In [71]:
# OR ...

In [72]:
df.loc[df.OR=='-', 'OR'] = pd.NA
df.OR = df.OR.astype(float)

In [73]:
# Comments ... as is

In [74]:
# CHECK columns which still have missing
# and dtypes
fColumns(df)
# RaceId dtype: -> string
# CourseName dtype: -> string
# Allow dtype: int64 ??
# OR etc dtype: float64 ??
# Going1 dtype: -> string
# ReturnWin, ReturnPlace NA <- Sp NA ??

# beware implicit casts and overflow for int8, int16 columns

         column           dtype  nmissing
0        RaceId           int32         0
1    CourseName          object         0
2      RaceDate          object         0
3      RaceTime          object         0
4          Type  string[python]    656127
5         Class         float64    270942
6         Prize         float64         0
7     FieldSize            int8         0
8      Distance           int16         0
9   RaceWinTime         float64       164
10         Posn  string[python]         0
11     TotalBtn         float32     65378
12    HorseName  string[python]         0
13         Draw         float64    429890
14           Sp         float64        22
15    WeightLBS           int16         0
16          Aid  string[python]    682621
17      Trainer  string[python]         0
18       Jockey  string[python]         0
19        Allow           int64         0
20           OR         float64    282022
21     Comments  string[python]       494
22    Territory          object   

In [75]:
# TODO
# save to csv once rc1

In [76]:
df.to_csv(fpath+'form_data_v2.0.csv',header=True,index=False)